# Train a Classification Model

In this notebook we create and train a model which is assumed to infer on 'AI Inference Server'.  
The model itself is the well known `Mobilnet` classifier network pre-trained for images, and at the end of this notebook we extend it with layers and train it to be able to classify our example images.  
The notebook [20-CreateInferenceWrapper](20-CreateInferenceWrapper.ipynb) will show how to wrap the model into Python code and which in- and outputs should be handled.  
The notebooks with number 30 will show how to create an 'Edge Configuration Package' to be imported directly on 'AI Inference Server'.

In [ ]:
import sys
from pathlib import Path
import tensorflow as tf

%matplotlib inline
print('tensorflow: {}'.format(tf.__version__))

sys.path.append('../src')
import utils

PYTHON_VERSION = sys.version_info


### Download dataset 

To train a model, an image set is required.  
Such an image set is available in the repository of [Simatic AI Launcher](https://github.com/siemens/simatic-ai-launcher).
In the next cell, and the `tf.keras.utils.get_file(..)` command will download and extract the tar-zipped file into the `../data/processed` folder.

In [ ]:
file_name = tf.keras.utils.get_file(
    fname=Path('../data/raw/simatic_photos.tgz').resolve(),
    origin='https://github.com/siemens/simatic-ai-launcher/raw/master/datasets/simatic_photos.tgz',
    extract=True, cache_dir=Path('../data/processed/').resolve(), cache_subdir="")

## Retrain an existing Convolutional Neural Network

### Create a dataset for training

The `../data/processed/simatic_photos` folder is created in the previous step and contains one folder for every class to be identified. The images belonging to a class are within the corresponding folder.  

In [ ]:
image_dir = Path('../data/processed/simatic_photos')

class_labels = [path.name for path in image_dir.iterdir() if path.is_dir()]
class_labels.sort()

In [ ]:
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_SIZE = (IMAGE_WIDTH,IMAGE_HEIGHT)
SCALE = 255

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/SCALE, validation_split=0.2)
training_set = image_generator.flow_from_directory(str(image_dir), target_size=IMAGE_SIZE, subset='training')
validation_set = image_generator.flow_from_directory(str(image_dir), target_size=IMAGE_SIZE, subset='validation')

### Retrieve MobileNet network

In [ ]:
# in this command result deprecation warning can be shown up, but it is normal
dim = 3
feature_extractor = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE+(dim,))) 
feature_extractor.trainable=False
print(f'Type of feature extractor: {type(feature_extractor)}')

### Build the model 

In [ ]:
image_batch, label_batch = next(iter(training_set))

x = feature_extractor.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# Uncomment to increase accuracy with additional hidden alyers:
# x = tf.keras.layers.Dense(label_batch.shape[1] ** 2, activation='relu')(x)
classifier = tf.keras.layers.Dense(label_batch.shape[1], activation='softmax')(x) # final layer with softmax activation

model = tf.keras.Model(inputs=feature_extractor.input,outputs=classifier)
    
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

### Prepare for training

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

### Train the model

In [ ]:
history = model.fit(training_set, epochs=4,
                              steps_per_epoch=training_set.num_batches,
                              validation_data=validation_set,
                              callbacks = [])

### Check predictions

Now we are able to check the accuracy of the trained model by plotting its confusion matrix.  
Please note that this plot only checks the first batch of the validation set.  
If everything went well, we are done, and we have a trained model we can use as the neural network in the AI Template.

In [ ]:
image_batch, label_batch = validation_set[0]
predictions = model.predict(image_batch)

utils.show_confusion(label_batch, predictions, class_labels)

### Save model

This model can be used on TM-NPU and on AI Inference Server with TensorFlow as well.

In [ ]:
model.save(Path('../models/classification_mobilnet.h5'))

### Convert model to TensorFlow lite

This version of the model can be used on AI Inference Server with TFLite and results in a more lightweight pipeline.

Please note, there is no official TFLite runtime for Windows.

You can either use the full TensorFlow package, or build the TFLite wheel manually, following this guide: https://www.tensorflow.org/lite/guide/build_cmake_pip

In [ ]:
import tensorflow as tf2

# Convert the model.
converter = tf2.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('../models/classification_mobilnet.tflite', 'wb') as f:
  f.write(tflite_model)

The saved model can be used in notebook [30-CreatePipelinePackage](30-CreatePipelinePackage.ipynb) to create a deployment package.